In [85]:
import re
import wordninja   
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import en_core_web_lg  
from itertools import groupby
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [271]:
G = nx.read_gpickle('HealthApp_Bigram.gpickle')

In [211]:
def log_transformer(pdoc):
    #https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
    countVec = CountVectorizer(max_features= 5000, stop_words='english', min_df=.01, max_df=.90)

    #%%
    #use CountVectorizer.fit(self, raw_documents[, y] to learn vocabulary dictionary of all tokens in raw documents
    #raw documents in this case will betweetsFrameWords["Text"] (processed text)
    countVec.fit(pdoc)
    #useful debug, get an idea of the item list you generated
    list(countVec.vocabulary_.items())

    #%%
    #convert to bag of words
    #sparse matrix representation? (README: could use an edit/explanation)
    countVec_count = countVec.transform(pdoc)

    #%%
    #make array from number of occurrences
    occ = np.asarray(countVec_count.sum(axis=0)).ravel().tolist()

    #make a new data frame with columns term and occurrences, meaning word and number of occurences
    bowListFrame = pd.DataFrame({'term': countVec.get_feature_names(), 'occurrences': occ})
    # print(bowListFrame)

    #sort in order of number of word occurences, most->least. if you leave of ascending flag should default ASC
    bowListFrame.sort_values(by='occurrences', ascending=False).head(60)

    #%%
    #now, convert to a more useful ranking system, tf-idf weights
    #TfidfTransformer: scale raw word counts to a weighted ranking using the
    #https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
    logTransformer = TfidfTransformer()
    logTransformer.fit_transform(countVec_count)

    df_idf = pd.DataFrame(logTransformer.idf_,index=countVec.get_feature_names(),columns=["idf_weights"]) 
    # sort ascending 
    df_idf= df_idf.sort_values(by=['idf_weights'])
    # print (df_idf.head(30))
    return df_idf

TDIFD to connected compponents of graph to find relevant tags

In [212]:
S = [G.subgraph(c) for c in nx.connected_components(G)]  
print(len(S)  )
tags_cluster = []
nlp =en_core_web_lg.load()
for cluster in  S:
    
    if len(cluster)> 3:
        df = log_transformer(list(cluster.nodes()))
        tags_list = dict(df.iloc[:,0]).keys()

        tags_list = nlp(' '.join(tags_list))
        tags_cluster =  [token.lemma_ for token in tags_list if token not in nlp.Defaults.stop_words]
        nx.set_node_attributes(cluster, tags_cluster , 'tags')

    else:
        string =' '.join (cluster.nodes())
        cleanString = re.sub('\W+',' ', string )
        doc = nlp(cleanString)
        tags_cluster = [token.lemma_ for token in doc if token not in nlp.Defaults.stop_words]
        nx.set_node_attributes(cluster, tags_cluster , 'tags')

print(G.nodes['Verification succeeded for <*>']['tags'])

1
['root', 'task', 'user', 'allocate', 'temporary', 'rand', 'size', 'datum', 'oop', 'receive', 'file', 'delete', 'current', 'sort', 'add', 'delete', 'txt', 'ask', 'start', 'store', 'sub', 'add', 'thread', 'transfer', 'update', 'succeed', 'terminate', 'receive', 'serve', 'serve', 'responder', 'replicate', 'packet', 'node', 'map', 'invalid', 'get', 'exception', 'emporary', 'dir', 'set', 'verification']


Adding a new node using TFIDF

In [240]:

def remove_repeated(val):
  x = val.split(" ")
  str_list = list(filter(None, x))
  clean = [i[0] for i in groupby(str_list)]
  return ' '.join(clean)


def preprocess_document(document):
    dic_debug = {}
    # log_list =  [re.sub(r"[^A-Za-z]+", ' ', _) for _ in document]
    for log in document:
        splitted_log = wordninja.split(log)
        remove_urls = re.sub('[^ ]+\.[^ ]+','<*>',' '.join(splitted_log) )
        
        remove_float = re.sub("(\d*\.\d+)|(\d+\.[0-9 ]+)", "<*>", remove_urls) 
        remove_int = re.sub(r'\d+', "<*>", remove_float )
        generic_str = re.sub(r'\b\w{1,2}\b', '', remove_int) 

        dic_debug[log]  = remove_repeated(generic_str)
    logs_clean =dic_debug.values()
    return logs_clean

In [272]:
atrributes = Counter(list(G.nodes()))
nx.set_node_attributes(G, atrributes, "count")

In [274]:

def add_node_if_exist(G,node):
    
    
    relationship = []
    nodeset =(G.nodes())
    if node in nodeset:
        count = G.nodes[node]['count']
        print(node, G.nodes[node])
        count +=1
        print("node already in graph", node, count)
        nx.set_node_attributes(G, {node : count}, 'count')
        count = G.nodes[node]['count']
    else:
        print("in get relation step ", node)
        S = [G.subgraph(c) for c in nx.connected_components(G)] 
        cleanString = node.replace("<*>", "")
        tags = set(cleanString.split())
        for subg in S :
            tags_dict = nx.get_node_attributes(subg, "tags")
            keydict = next(iter(tags_dict),None)
            if keydict and bool(tags.intersection(tags_dict[keydict])):
                node_scores = nx.betweenness_centrality(subg)
                sorted_node_scores = dict(sorted(node_scores.items(), key=lambda item: -item[1]))
                sorted_nodes = list(sorted_node_scores.keys())
                pair = [cleanString,list(sorted_nodes[0] )]
                G.add_node(cleanString)
                G.add_edges_from(relationship)
                print("node succefully added as " + cleanString)
                return cleanString
            else:
                print("cannot find edges to node ->" , node)
                return 0


In [276]:
#read csv file to list

df = pd.read_csv('results/HDFS mockdata.csv', sep=',',header=None)
# print(df.head())
new_nodes = df[0].tolist()  
# print(nx.get_node_attributes(G, "tags"))
p_node = preprocess_document(new_nodes)
print(len(p_node))
start = pd.Timestamp.now()
for node in p_node:
    t =add_node_if_exist(G,node)
    if t:   
        print(G[t])
    else:
        print("node not added")
print(pd.Timestamp.now()-start)


342
in get relation step  Receiving block <*>
cannot find edges to node -> Receiving block <*>
node not added
in get relation step  Starting thread transfer block <*>
cannot find edges to node -> Starting thread transfer block <*>
node not added
in get relation step  BLOCK Name System add Stored Block block Map updated added <*> size <*>
cannot find edges to node -> BLOCK Name System add Stored Block block Map updated added <*> size <*>
node not added
in get relation step  BLOCK Name System allocate Block user root rand txt <*> temporary task <*> part <*>
cannot find edges to node -> BLOCK Name System allocate Block user root rand txt <*> temporary task <*> part <*>
node not added
in get relation step  BLOCK Name System allocate Block user root rand <*> empor ary task <*> part <*>
cannot find edges to node -> BLOCK Name System allocate Block user root rand <*> empor ary task <*> part <*>
node not added
in get relation step  BLOCK Name System allocate Block user root sort rand <*> tempo